In [1]:
!pip install -qq transformers

In [2]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
import pandas as pd

In [3]:
df = pd.read_csv("IMDB_128_2000_.csv")
df.head()

,review,score
0,I had the privilege of attending a regional pr...,1
1,Powers Booth is hypnotic as cult leader jim jo...,1
2,I so much enjoyed this little musical fantasy ...,1
3,This movie is utterly hilarious. Its cast clic...,1
4,Whoever made this movie must have done it as a...,0


In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [5]:
def bert_encode(reviews):
    encoded_input = tokenizer(reviews, padding=True, truncation=True, max_length=128, return_tensors='pt')
    with torch.no_grad():
        output = model(**encoded_input)
    return output.last_hidden_state.mean(1).numpy()

# Векторизация отзывов
reviews_texts = df['review'].tolist()
X = bert_encode(reviews_texts)

In [7]:
# Кластеризация
num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
clusters = kmeans.fit_predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [8]:
# Добавление меток кластеров к DataFrame
clusters = kmeans.labels_
df['cluster_labels'] = clusters

df.to_csv('reviews_with_clusters.csv', index=False)

In [9]:
from sklearn.metrics import accuracy_score, f1_score

In [12]:
# Оценка
accuracy = accuracy_score(df['score'], df['cluster_labels'])
f1 = f1_score(df['score'], df['cluster_labels'], average='weighted')

print(f"Точность: {accuracy}")
print(f"F1: {f1}")

Точность: 0.4695
F1: 0.4693555320435988
